# Final Project - IR

Name: Wenjie Jin <br>
NIA: 219516

https://developer.twitter.com/en/apps

In [15]:
#Import librery
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
from collections import Counter
from config import *

#Import the necessary methods from tweepy library
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import API
from tweepy import Cursor
import json
import datetime


In [23]:
## access token informations 
access_token1 = "1240651902712516613-BF3CepSxPUoMr3lDkptNt08f2GwaGZ"
access_token_secret1 = "79ga9EksEgtgpHGJhXy7A4ujb31pw1AjwP8lOlaYtmDdk"

consumer_key1 = "ieusj8Wof83kmsVe9PDLJQqn3"
consumer_secret1 = "EpZYsMfEqvHSwFQyAfz2wCdVujPTtr7JQtvkBt1g9sMg5t0m3B"

In [24]:
#Generate API
auth = OAuthHandler(consumer_key1, consumer_secret1)
auth.set_access_token(access_token1, access_token_secret1)
api = API(auth_handler=auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

# class MyStreamListener(StreamListener):
    """
    Twitter listener, collects streaming tweets and output to a file
    """
    def __init__(self, api, OUTPUT_FILENAME, stop_condition=10):
        """
        initialize the stream, with num. of tweets and saving the outputfile
        """
        
        # this line is needed to import the characteristics of the streaming API
        super(MyStreamListener, self).__init__()
        
        # to-count the number of tweets collected        
        self.num_tweets = 0
        
        # save filename
        self.filename = OUTPUT_FILENAME
        
        # stop-condition
        self.stop_condition = stop_condition
        

    def on_status(self, status):
        
        """
        this function runs each time a new bunch of tweets is retrived from the streaming
        """
        
        with open(self.filename, "a+") as f:
            tweet = status._json
            
            f.write(json.dumps(tweet) + '\n')
            #self.output.append(tweet)
            self.num_tweets += 1
        
            # Stop condition        
            if self.num_tweets <= self.stop_condition:
                return True
            else:
                return False
        

    def on_error(self, status):
        """
        function useful to handle errors. It's possible to personalize it 
        depending on the way we want to handle errors
        """
        
        print(status)
        #returning False in on_error disconnects the stream
        return False

##### Store File in Json. 

In [26]:
%%time

OUTPUT_FILENAME = "../other-outputs/output.json"
stop_condition = 10000

l = MyStreamListener(api, OUTPUT_FILENAME, stop_condition)
# here we recall the Stream Class from Tweepy to input the authentication info and our personalized listener 
stream = Stream(auth=api.auth, listener=l)


# keywords we may want decide to track 
TRACKING_KEYWORDS = ["coronavirus", "covid", "#COVID", "#coronavirus", "#COVID19", "recovery cases", "suspected cases", "death cases", "vaccines", "disinfection"]
stream.filter(
    track=TRACKING_KEYWORDS, 
    is_async=False, 
    languages = ["en"]
)

ProtocolError: ('Connection broken: IncompleteRead(7623 bytes read)', IncompleteRead(7623 bytes read))

##### Load data from Json

In [ ]:
#load data from output.json scrapped from twitter
with open("../other-outputs/output.json", "rb") as f:
    data = f.readlines()
    data = [json.loads(str_) for str_ in data]

df_tweets = pd.DataFrame.from_records(data)
df_tweets

###### Show tweets collection information

In [ ]:
def print_retweets_uniqueTweets_uniqueUsers(df_tweets):
    retweets_ = df_tweets["retweeted_status"].apply(lambda x: 0 if str(x) == "nan" else 1)
    tot_retweets = sum(retweets_)
    unique_tweets = len(retweets_) - tot_retweets
    
    users_ = df_tweets.user.apply(lambda x: x["id"])
    
    tot_users = len(set(users_))
    
    print("Total Retweets:", tot_retweets)

    print("Unique Tweets:", unique_tweets)
    
    print("Unique Users:", tot_users)

In [ ]:
print_retweets_uniqueTweets_uniqueUsers(df_tweets)

In [ ]:
# Get unique tweets
tweets=df_tweets[df_tweets["retweeted_status"].isna()]
df_tweets.head()